<a href="https://colab.research.google.com/github/erikcferreira/Bitcoin_Fear_Greed_Analysis/blob/main/Bitcoin_FearGreed_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
Bitcoin Fear Greed Index Dashboard
Author: Erik Ferreira
Date: 01/10/2025
"""

#yfinance was chosen by its simplicity. For more robust applications try Alpha Vantage or CCXT.

!pip install yfinance

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests

In [3]:
#First create the bitcoin-dollar object, than call its history.

btc_usd = yf.Ticker("BTC-USD")

In [4]:
btc_data = btc_usd.history(period="max")

In [5]:
print(type(btc_usd))
print(type(btc_data))
btc_data.head()

<class 'yfinance.ticker.Ticker'>
<class 'pandas.core.frame.DataFrame'>


,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2014-09-17 00:00:00+00:00,465.864014,468.174011,452.421997,457.334015,21056800,0.0,0.0
2014-09-18 00:00:00+00:00,456.859985,456.859985,413.104004,424.440002,34483200,0.0,0.0
2014-09-19 00:00:00+00:00,424.102997,427.834991,384.532013,394.795990,37919700,0.0,0.0
2014-09-20 00:00:00+00:00,394.673004,423.295990,389.882996,408.903992,36863600,0.0,0.0
2014-09-21 00:00:00+00:00,408.084991,412.425995,393.181000,398.821014,26580100,0.0,0.0


In [6]:
#Get the Fear and Greed data from Alternative.me using Requests library.
#Use .../?limit=0 to get everything availeble.

In [7]:
url = "https://api.alternative.me/fng/?limit=0"

In [8]:
feargreed_data = requests.get(url).json()

In [9]:
#Just to check it out what the JSON has

print(feargreed_data.keys())

dict_keys(['name', 'data', 'metadata'])


In [10]:
#Exploring the JSON file

for key in feargreed_data.keys():
    print(f"--- {key} ---")

    value = feargreed_data[key]

    # Se for lista, pega o primeiro elemento
    if isinstance(value, list):
        print("Primeira linha:", value[0])

    # Se for dict, mostra os pares chave/valor
    elif isinstance(value, dict):
        print("Conteúdo:", value)

    # Se for valor simples (string, número, etc.)
    else:
        print("Valor:", value)


--- name ---
Valor: Fear and Greed Index
--- data ---
Primeira linha: {'value': '29', 'value_classification': 'Fear', 'timestamp': '1768089600', 'time_until_update': '40457'}
--- metadata ---
Conteúdo: {'error': None}


In [11]:
#Create a FearGreed DataFrame

fg_df = pd.DataFrame(feargreed_data["data"])

fg_df.head()

,value,value_classification,timestamp,time_until_update
0,29,Fear,1768089600,40457
1,25,Extreme Fear,1768003200,NaN
2,27,Fear,1767916800,NaN
3,28,Fear,1767830400,NaN
4,42,Fear,1767744000,NaN


In [12]:
#Cleaning the DataFrame

fg_df["date"] = pd.to_datetime(fg_df['timestamp'], unit='s')

fg_df = fg_df.set_index('date')

fg_df = fg_df.sort_index()

fg_df = fg_df.drop(['timestamp','time_until_update'], axis=1)

fg_df.head()

/tmp/ipython-input-1623184987.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  fg_df["date"] = pd.to_datetime(fg_df['timestamp'], unit='s')


,value,value_classification
date,,
2018-02-01,30,Fear
2018-02-02,15,Extreme Fear
2018-02-03,40,Fear
2018-02-04,24,Extreme Fear
2018-02-05,11,Extreme Fear


In [13]:
fg_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2898 entries, 2018-02-01 to 2026-01-11
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   value                 2898 non-null   object
 1   value_classification  2898 non-null   object
dtypes: object(2)
memory usage: 67.9+ KB


In [14]:
# let's convert any integers that are strings into intergers

fg_df["value"] = pd.to_numeric(fg_df["value"])

fg_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2898 entries, 2018-02-01 to 2026-01-11
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   value                 2898 non-null   int64 
 1   value_classification  2898 non-null   object
dtypes: int64(1), object(1)
memory usage: 67.9+ KB


In [15]:
#Merging the data from left to keep all the Bitcoin history
#The btc_data DataFrame has a different timezone conffiguration (timezone-aware), We need to remove it.

btc_data.index = btc_data.index.tz_localize(None)

fg_btc_df = btc_data.merge(fg_df, how="left", left_index=True, right_index=True)


In [16]:
fg_btc_df.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,value,value_classification
Date,,,,,,,,,
2026-01-07,93727.468750,93738.789062,90601.804688,91308.054688,43461295053,0.0,0.0,42.0,Fear
2026-01-08,91309.640625,91485.851562,89233.875000,91027.125000,42386697030,0.0,0.0,28.0,Fear
2026-01-09,91026.273438,91910.671875,89625.382812,90513.101562,38305906684,0.0,0.0,27.0,Fear
2026-01-10,90510.101562,90713.031250,90283.398438,90386.648438,12385895282,0.0,0.0,25.0,Extreme Fear
2026-01-11,90385.625000,90738.578125,90340.718750,90731.671875,12179481600,0.0,0.0,29.0,Fear


In [17]:
fg_btc_df = fg_btc_df.drop(['Dividends','Stock Splits'], axis=1 )
fg_btc_df.head()

,Open,High,Low,Close,Volume,value,value_classification
Date,,,,,,,
2014-09-17,465.864014,468.174011,452.421997,457.334015,21056800,NaN,NaN
2014-09-18,456.859985,456.859985,413.104004,424.440002,34483200,NaN,NaN
2014-09-19,424.102997,427.834991,384.532013,394.795990,37919700,NaN,NaN
2014-09-20,394.673004,423.295990,389.882996,408.903992,36863600,NaN,NaN
2014-09-21,408.084991,412.425995,393.181000,398.821014,26580100,NaN,NaN


In [18]:
fg_btc_df.tail()

,Open,High,Low,Close,Volume,value,value_classification
Date,,,,,,,
2026-01-07,93727.468750,93738.789062,90601.804688,91308.054688,43461295053,42.0,Fear
2026-01-08,91309.640625,91485.851562,89233.875000,91027.125000,42386697030,28.0,Fear
2026-01-09,91026.273438,91910.671875,89625.382812,90513.101562,38305906684,27.0,Fear
2026-01-10,90510.101562,90713.031250,90283.398438,90386.648438,12385895282,25.0,Extreme Fear
2026-01-11,90385.625000,90738.578125,90340.718750,90731.671875,12179481600,29.0,Fear


In [19]:
fg_btc_df = fg_btc_df.rename(columns={"Close": "BTC_Price_USD", "value": "Fear_Greed_Index", 'value_classification': "Sentiment_Label"})
fg_btc_df.tail()

,Open,High,Low,BTC_Price_USD,Volume,Fear_Greed_Index,Sentiment_Label
Date,,,,,,,
2026-01-07,93727.468750,93738.789062,90601.804688,91308.054688,43461295053,42.0,Fear
2026-01-08,91309.640625,91485.851562,89233.875000,91027.125000,42386697030,28.0,Fear
2026-01-09,91026.273438,91910.671875,89625.382812,90513.101562,38305906684,27.0,Fear
2026-01-10,90510.101562,90713.031250,90283.398438,90386.648438,12385895282,25.0,Extreme Fear
2026-01-11,90385.625000,90738.578125,90340.718750,90731.671875,12179481600,29.0,Fear


In [20]:
# Save to CSV
fg_btc_df.to_csv("Bitcoin_Fear_Greed_Dataset.csv")

In [21]:
from google.colab import files
files.download("Bitcoin_Fear_Greed_Dataset.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>